In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *
from fastai.metrics import *
from fastai.model import *

import torch
import torch.nn as nn

['train-images-idx3-ubyte', 'train-labels-idx1-ubyte', 't10k-labels-idx1-ubyte', 'fashion-mnist_train.csv', 't10k-images-idx3-ubyte', 'fashion-mnist_test.csv']


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
trn_path, tst_path='../input/fashion-mnist_train.csv', '../input/fashion-mnist_test.csv'
trn_data, tst_data = pd.read_csv(trn_path), pd.read_csv(tst_path)
trn_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [3]:
x_raw, y_raw = trn_data.drop(['label'], axis=1).values, trn_data['label'].values
x_raw.shape

(60000, 784)

In [4]:
mean = x_raw.mean()
std = x_raw.std()

In [5]:
x_raw = (x_raw-mean)/std

In [6]:
from sklearn.model_selection import train_test_split
x_trn, x_valid, y_trn, y_valid = train_test_split(x_raw, y_raw, test_size=0.2, random_state=1)

In [7]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

In [8]:
from fastai.dataset import *
md = ImageClassifierData.from_arrays('tmp/tmp/', (x_trn,y_trn), (x_valid, y_valid))

In [9]:
loss=nn.NLLLoss()
metrics=[accuracy]
# opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9)
opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In [10]:
fit(net, md, n_epochs=10, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                     
    0      0.574885   0.604147   0.76125   
    1      0.519384   0.576206   0.819167                     
    2      0.556145   0.581179   0.81725                      
    3      0.549828   0.52111    0.826833                     
    4      0.559562   0.583834   0.812083                     
    5      0.53661    0.540247   0.812333                     
    6      0.55081    0.489568   0.82375                      
    7      0.559917   0.604152   0.77675                      
    8      0.547902   0.530576   0.826333                     
    9      0.558335   0.523477   0.801333                     



[0.523477058728536, 0.8013333333333333]

In [11]:
set_lrs(opt, 0.0005)

fit(net, md, n_epochs=15, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                     
    0      0.383775   0.426283   0.843833  
    1      0.365208   0.414234   0.851                        
    2      0.372976   0.405731   0.85575                      
    3      0.371352   0.40039    0.85675                      
    4      0.364378   0.396712   0.857                        
    5      0.371283   0.392983   0.858917                     
    6      0.351044   0.389918   0.859667                     
    7      0.368708   0.387285   0.860833                     
    8      0.355307   0.385753   0.861333                     
    9      0.345108   0.382924   0.862417                     
    10     0.350443   0.381665   0.8625                       
    11     0.334524   0.379348   0.863833                     
    12     0.351922   0.37769    0.86325                      
    13     0.34644    0.375962   0.862917                     
    14     0.341947   0.374741   0.86425                      



[0.37474104086558024, 0.86425]